<a href="https://colab.research.google.com/github/mori01-22/kennkyu/blob/main/Sample1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
a = 5
b = 3
print("足し算の結果:", a + b)
print("引き算の結果:", a - b)
print("掛け算の結果:", a * b)
print("割り算の結果:", a / b)


足し算の結果: 8
引き算の結果: 2
掛け算の結果: 15
割り算の結果: 1.6666666666666667
